In [1]:
from math import sqrt
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline


In [2]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings_sample.csv')

In [4]:
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df[
    'title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

C:\Users\Saman\AppData\Local\Temp\ipykernel_12336\863525776.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  'title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [6]:
movies_df.head()


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


### دراپ کن ژانر هارو چون اصن ژانر برام مهم نیست

In [7]:
#Dropping the genres column
movies_df = movies_df.drop('genres', 1)

C:\Users\Saman\AppData\Local\Temp\ipykernel_12336\37422046.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  movies_df = movies_df.drop('genres', 1)


### شکل و شیپ دیتاست مویز که ژانر ها حذف گردیده هست.

In [12]:
movies_df.head()
# movies_df.shape

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [11]:
ratings_df.head()


,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


### دراپ کردن ویژگی تایم از دیتاست ریتینگ و امتیازی

In [14]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

C:\Users\Saman\AppData\Local\Temp\ipykernel_12336\1971122656.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_df = ratings_df.drop('timestamp', 1)


In [15]:
ratings_df.head()


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


### مشخصات افراد و فیلم های ریتینگ شده

In [16]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


### اون فیلم هایی که دوست داشته را جدا کن + حذف سال از دیتاست

In [17]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

C:\Users\Saman\AppData\Local\Temp\ipykernel_12336\2035672847.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('year', 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


### آدم های شبیه به اون فردی که آن فیلم رو دیده 

In [20]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()
# userSubset.shape

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


### یوزر هارو جدا کردیم . تمام ریت هایی که یوزر ها داده اند نسبت به فیلم هایی

In [21]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

In [22]:
userSubsetGroup.get_group(13)


,userId,movieId,rating
479,13,2,2.0
531,13,1274,5.0


### مرتب کردن آدم ها بر حسب تعداد فیلم مشترک 

In [23]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [25]:
userSubsetGroup[0:5]


[(75,
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 (106,
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 (686,
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0),
 (815,
         userId  movieId  rating
  73747     815        1     4.5
  73748     815        2     3.0
  73922     815      296     5.0
  74362     815     1274     3.0
  74678     815     1968     4.5),
 (1040,
         userId  movieId  rating
  96689    1040        1     3.0
  96690    1040        2     1.5
  96733    1040      296     3.5
  96859    1040     1274     3.0
  96922    1

### با استفاده از روشی فاصله بین آدمای مشترک رو حساب کرده.
###  محاسبه روش وابستگی پیرسون 

In [30]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [32]:
pearsonCorrelationDict


{75: 0.8272781516947562,
 106: 0.5860090386731182,
 686: 0.8320502943378437,
 815: 0.5765566601970551,
 1040: 0.9434563530497265,
 1130: 0.2891574659831201,
 1502: 0.8770580193070299,
 1599: 0.4385290096535153,
 1625: 0.716114874039432,
 1950: 0.179028718509858,
 2065: 0.4385290096535153,
 2128: 0.5860090386731196,
 2432: 0.1386750490563073,
 2791: 0.8770580193070299,
 2839: 0.8204126541423674,
 2948: -0.11720180773462392,
 3025: 0.45124262819713973,
 3040: 0.89514359254929,
 3186: 0.6784622064861935,
 3271: 0.26989594817970664,
 3429: 0.0,
 3734: -0.15041420939904673,
 4099: 0.05860090386731196,
 4208: 0.29417420270727607,
 4282: -0.4385290096535115,
 4292: 0.6564386345361464,
 4415: -0.11183835382312353,
 4586: -0.9024852563942795,
 4725: -0.08006407690254357,
 4818: 0.4885967564883424,
 5104: 0.7674257668936507,
 5165: -0.4385290096535153,
 5547: 0.17200522903844556,
 6082: -0.04728779924109591,
 6207: 0.9615384615384616,
 6366: 0.6577935144802716,
 6482: 0.0,
 6530: -0.351605423203

### مرتب کردن و ساخت دیتا فریم بر اساس یوزر  آیدی و ایندکس شباهت

In [34]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.827278,75
1,0.586009,106
2,0.832050,686
3,0.576557,815
4,0.943456,1040


###  مرتب کردن نزولی بر اساس شباهت بین آدما و یوزر آیدی شون 

In [36]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
2369,1.0,20192
2850,1.0,29143
3496,1.0,40611
1709,1.0,8902
3483,1.0,40314


### ادغام کردن دوتا دیتا فریم براساس تاپ یورزام

In [38]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,20192,1,3.5
1,1.0,20192,31,3.0
2,1.0,20192,32,5.0
3,1.0,20192,47,5.0
4,1.0,20192,110,3.0


In [39]:
topUsersRating.shape

(18557, 4)

In [41]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,20192,1,3.5,3.5
1,1.0,20192,31,3.0,3.0
2,1.0,20192,32,5.0,5.0
3,1.0,20192,47,5.0,5.0
4,1.0,20192,110,3.0,3.0


In [42]:
topUsersRating.shape

(18557, 5)

In [43]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,43.0,144.5
2,7.0,20.0
3,6.0,16.0
4,4.0,6.5
5,8.0,18.5


### استفاده از فرمول حاصل جمع وزنی علاقه مندی نسبت به یک فیلم تقسیم بر میزان شباهت آدما

In [44]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.360465,1
2,2.857143,2
3,2.666667,3
4,1.625000,4
5,2.312500,5


### مرتب  سازی فیلم ها بر اساس امتیازی که کاربر ما به این فیلم ها احتمالا بدهد

In [45]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
3548,5.0,3548
1785,5.0,1785
1964,5.0,1964
3549,5.0,3549
97950,5.0,97950
3551,5.0,3551
3604,5.0,3604
48744,5.0,48744
314,5.0,314


### اسم ۱۰ فیلم برتری که بنابر امتیاز احتمالی به کاربر پیشنهاد میدهد

In [46]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]


,movieId,title,year
311,314,"Secret of Roan Inish, The",1994
1714,1785,King of New York,1990
1881,1964,Klute,1971
3458,3548,Auntie Mame,1958
3459,3549,Guys and Dolls,1955
3461,3551,Marathon Man,1976
3514,3604,Gypsy,1962
3528,3618,Small Time Crooks,2000
11408,48744,Shortbus,2006
19876,97950,"Man with the Iron Fists, The",2012
